# SEIRX model runs for test technology and screening frequency investigation

In [1]:
import pandas as pd
from os.path import join
import json

# parallelisation functionality
from multiprocess import Pool
import psutil
from tqdm import tqdm

from scseirx.model_nursing_home import SEIRX_nursing_home # agent-based simulation
import data_creation_functions as dcf

In [2]:
# paths for data I/O
contact_network_src = '../data/contact_networks'

## Simulation function

In [3]:
def run(param_list):
    '''
    Runs an ensemble of simulations and collects observable statistics. To be 
    run in parallel on many workers. Note: I/O paths and the number of runs per 
    ensemble hare hard coded here, because I only want to pass the parameter 
    values that are being screened in the simulation run to the function via the
    parallel processing interface.
    
    Parameters:
    -----------
    param_list : iterable
        Iterable that contains the values for the parameters test_type, 
        index_case, e_screen_range and r_screen_range that are passed to the
        simulation.
    N_runs : integer
        Number of runs per ensemble.
        
    Returns:
    --------
    row : dictionary
        Dictionary of the ensemble statistics of the observables.
    '''

    # extract the simulation parameters from the parameter list
    test_type, index_case, e_screen_interval, r_screen_interval, \
        e_vaccination_ratio, r_vaccination_ratio, N_runs = param_list

    # run the ensemble with the given simulation parameters 
    ensemble_results = dcf.run_ensemble(N_runs, measures, simulation_params, 
                contact_network_src, ensmbl_dst, test_type, index_case,
                e_screen_interval=e_screen_interval,
                r_screen_interval=r_screen_interval,
                e_vaccination_ratio=e_vaccination_ratio,
                r_vaccination_ratio=r_vaccination_ratio,
               )

    # calculate the ensemble statistics
    row = dcf.evaluate_ensemble(ensemble_results, index_case, 
            test_type=test_type, e_screen_interval=e_screen_interval,
            r_screen_interval=r_screen_interval,
            e_vaccination_ratio=e_vaccination_ratio,
            r_vaccination_ratio=r_vaccination_ratio)

    return row

## Screening parameters

In [4]:
#test_types = ['same_day_antigen', 'same_day_LAMP']
test_types = ['same_day_antigen', 'same_day_PCR']
vaccination_scenarios = [(0.0, 0.5), (0.5, 0.0), (0.0, 0.9), (0.5, 0.9), (0.9, 0.9)]

# resident and employee streening intervals (in days)
e_screen_range = [2, 3, 7, None]
r_screen_range = [2, 3, 7, None]

# specifies, whether the index case will be introduced via an
# employee or a resident
index_cases = ['employee', 'resident']

# ensemble_size
N_runs = 5000

screening_params = [(i, j, k, l, m, n, N_runs)\
              for i in test_types \
              for j in index_cases \
              for k in e_screen_range \
              for l in r_screen_range \
              for m, n in vaccination_scenarios]

print('there are {} parameter combinations'.format(len(screening_params)))

there are 320 parameter combinations


## B.1.1.7

### Simulation parameters and measures in place

In [5]:
with open('params/testing_and_vaccination_UK_variant_simulation_parameters.json') as json_file:
    simulation_params = json.load(json_file)
    
with open('params/testing_and_vaccination_measures.json') as json_file:
    measures = json.load(json_file)


In [6]:
simulation_params

{'verbosity': 0,
 'base_transmission_risk': 0.109549,
 'subclinical_modifier': 0.6,
 'exposure_duration': [5.0, 1.9],
 'time_until_symptoms': [6.4, 0.8],
 'infection_duration': [10.91, 3.95],
 'infection_risk_contact_type_weights': {'very_far': 0,
  'far': 0.13,
  'intermediate': 0.13,
  'close': 1},
 'employee_index_probability': 0,
 'resident_index_probability': 0,
 'age_transmission_discount': {'slope': 0, 'intercept': 1},
 'age_symptom_modification': {'slope': 0.009515, 'intercept': -0.116}}

In [7]:
measures

{'testing': 'preventive',
 'preventive_screening_test_type': None,
 'diagnostic_test_type': 'two_day_PCR',
 'K1_contact_types': ['close', 'intermediate'],
 'quarantine_duration': 10,
 'employee_mask': False,
 'resident_mask': False,
 'follow_up_testing_interval': None,
 'liberating_testing': False,
 'transmission_risk_ventilation_modifier': 1,
 'transmission_risk_vaccination_modifier': {'reception': 0.6,
  'transmission': 0.3},
 'mask_filter_efficiency': {'exhale': 0.5, 'inhale': 0.7}}

### Simulation runs

In [8]:
%%time
ensmbl_dst = '../data/simulation_results/ensembles/testing_and_vaccination/UK_variant'

number_of_cores = psutil.cpu_count(logical=True) - 3 
pool = Pool(number_of_cores)

#rows = pool.imap_unordered(run, screening_params)
rows = []
for row in tqdm(pool.imap_unordered(func=run, iterable=screening_params),
                total=len(screening_params)):
        rows.append(row)

# turn off your parallel workers 
pool.close()

# format and dump the results to disk
results = pd.DataFrame()
for row in rows:
    results = results.append(row, ignore_index=True)
    
results.reset_index()
index_cols = ['test_type', 'index_case', 'resident_screen_interval',
              'employee_screen_interval',
              'resident_vaccination_ratio', 'employee_vaccination_ratio']

other_cols = [c for c in results.columns if c not in index_cols]
results = results[index_cols + other_cols]

dst = '../data/simulation_results'
results.to_csv(join(dst,'simulations_testing_and_vaccination_UK_variant_{}.csv'\
                   .format(N_runs)), index=False)

results.head(3)

100%|██████████| 320/320 [3:04:01<00:00, 34.51s/it]  


CPU times: user 6.78 s, sys: 588 ms, total: 7.37 s
Wall time: 3h 4min 6s


,test_type,index_case,resident_screen_interval,employee_screen_interval,resident_vaccination_ratio,employee_vaccination_ratio,N_agents_0.025,N_agents_0.10,N_agents_0.25,N_agents_0.75,...,transmissions_std,undetected_infections_0.025,undetected_infections_0.10,undetected_infections_0.25,undetected_infections_0.75,undetected_infections_0.90,undetected_infections_0.975,undetected_infections_mean,undetected_infections_median,undetected_infections_std
0,same_day_antigen,employee,2.0,2.0,0.9,0.9,53.0,53.0,53.0,53.0,...,0.187468,0.0,0.0,0.0,0.0,0.0,1.0,0.0614,0.0,0.240918
1,same_day_antigen,employee,3.0,2.0,0.9,0.9,53.0,53.0,53.0,53.0,...,0.203910,0.0,0.0,0.0,0.0,0.0,1.0,0.0522,0.0,0.223350
2,same_day_antigen,employee,3.0,2.0,0.9,0.5,53.0,53.0,53.0,53.0,...,0.338155,0.0,0.0,0.0,0.0,0.0,1.0,0.0640,0.0,0.245593


## Wild type

### Simulation parameters and measures in place

In [9]:
with open('params/testing_and_vaccination_simulation_parameters.json') as json_file:
    simulation_params = json.load(json_file)
    
with open('params/testing_and_vaccination_measures.json') as json_file:
    measures = json.load(json_file)


In [10]:
simulation_params

{'verbosity': 0,
 'base_transmission_risk': 0.073033,
 'subclinical_modifier': 0.6,
 'exposure_duration': [5.0, 1.9],
 'time_until_symptoms': [6.4, 0.8],
 'infection_duration': [10.91, 3.95],
 'infection_risk_contact_type_weights': {'very_far': 0,
  'far': 0.13,
  'intermediate': 0.13,
  'close': 1},
 'employee_index_probability': 0,
 'resident_index_probability': 0,
 'age_transmission_discount': {'slope': 0, 'intercept': 1},
 'age_symptom_modification': {'slope': 0.009515, 'intercept': -0.116}}

In [11]:
measures

{'testing': 'preventive',
 'preventive_screening_test_type': None,
 'diagnostic_test_type': 'two_day_PCR',
 'K1_contact_types': ['close', 'intermediate'],
 'quarantine_duration': 10,
 'employee_mask': False,
 'resident_mask': False,
 'follow_up_testing_interval': None,
 'liberating_testing': False,
 'transmission_risk_ventilation_modifier': 1,
 'transmission_risk_vaccination_modifier': {'reception': 0.6,
  'transmission': 0.3},
 'mask_filter_efficiency': {'exhale': 0.5, 'inhale': 0.7}}

### Simulation runs

In [12]:
%%time
ensmbl_dst = '../data/simulation_results/ensembles/testing_and_vaccination/wild_type'

number_of_cores = psutil.cpu_count(logical=True) - 3 
pool = Pool(number_of_cores)

#rows = pool.imap_unordered(run, screening_params)
rows = []
for row in tqdm(pool.imap_unordered(func=run, iterable=screening_params),
                total=len(screening_params)):
        rows.append(row)

# turn off your parallel workers 
pool.close()

# format and dump the results to disk
results = pd.DataFrame()
for row in rows:
    results = results.append(row, ignore_index=True)
    
results.reset_index()
index_cols = ['test_type', 'index_case', 'resident_screen_interval',
              'employee_screen_interval',
              'resident_vaccination_ratio', 'employee_vaccination_ratio']

other_cols = [c for c in results.columns if c not in index_cols]
results = results[index_cols + other_cols]

dst = '../data/simulation_results'
results.to_csv(join(dst,'simulations_testing_and_vaccination_{}.csv'\
                   .format(N_runs)), index=False)

results.head(3)

100%|██████████| 320/320 [2:50:05<00:00, 31.89s/it]  


CPU times: user 6.94 s, sys: 445 ms, total: 7.38 s
Wall time: 2h 50min 10s


,test_type,index_case,resident_screen_interval,employee_screen_interval,resident_vaccination_ratio,employee_vaccination_ratio,N_agents_0.025,N_agents_0.10,N_agents_0.25,N_agents_0.75,...,transmissions_std,undetected_infections_0.025,undetected_infections_0.10,undetected_infections_0.25,undetected_infections_0.75,undetected_infections_0.90,undetected_infections_0.975,undetected_infections_mean,undetected_infections_median,undetected_infections_std
0,same_day_antigen,employee,3.0,2.0,0.5,0.0,53.0,53.0,53.0,53.0,...,0.660183,0.0,0.0,0.0,0.0,0.0,1.0,0.0714,0.0,0.263659
1,same_day_antigen,employee,2.0,2.0,0.9,0.0,53.0,53.0,53.0,53.0,...,0.402941,0.0,0.0,0.0,0.0,0.0,1.0,0.0584,0.0,0.235373
2,same_day_antigen,employee,3.0,2.0,0.9,0.9,53.0,53.0,53.0,53.0,...,0.161895,0.0,0.0,0.0,0.0,0.0,1.0,0.0584,0.0,0.234522


## B.1.1.7 & masks for employees

### Simulation parameters and measures in place

In [13]:
with open('params/testing_and_vaccination_UK_variant_simulation_parameters.json') as json_file:
    simulation_params = json.load(json_file)
    
with open('params/testing_and_vaccination_masks_measures.json') as json_file:
    measures = json.load(json_file)


In [14]:
simulation_params

{'verbosity': 0,
 'base_transmission_risk': 0.109549,
 'subclinical_modifier': 0.6,
 'exposure_duration': [5.0, 1.9],
 'time_until_symptoms': [6.4, 0.8],
 'infection_duration': [10.91, 3.95],
 'infection_risk_contact_type_weights': {'very_far': 0,
  'far': 0.13,
  'intermediate': 0.13,
  'close': 1},
 'employee_index_probability': 0,
 'resident_index_probability': 0,
 'age_transmission_discount': {'slope': 0, 'intercept': 1},
 'age_symptom_modification': {'slope': 0.009515, 'intercept': -0.116}}

In [15]:
measures

{'testing': 'preventive',
 'preventive_screening_test_type': None,
 'diagnostic_test_type': 'two_day_PCR',
 'K1_contact_types': ['close', 'intermediate'],
 'quarantine_duration': 10,
 'employee_mask': True,
 'resident_mask': False,
 'follow_up_testing_interval': None,
 'liberating_testing': False,
 'transmission_risk_ventilation_modifier': 1,
 'transmission_risk_vaccination_modifier': {'reception': 0.6,
  'transmission': 0.3},
 'mask_filter_efficiency': {'exhale': 0.5, 'inhale': 0.7}}

### Simulation runs

In [16]:
%%time
ensmbl_dst = '../data/simulation_results/ensembles/testing_and_vaccination/masks'

number_of_cores = psutil.cpu_count(logical=True) - 3 
pool = Pool(number_of_cores)

#rows = pool.imap_unordered(run, screening_params)
rows = []
for row in tqdm(pool.imap_unordered(func=run, iterable=screening_params),
                total=len(screening_params)):
        rows.append(row)

# turn off your parallel workers 
pool.close()

# format and dump the results to disk
results = pd.DataFrame()
for row in rows:
    results = results.append(row, ignore_index=True)
    
results.reset_index()
index_cols = ['test_type', 'index_case', 'resident_screen_interval',
              'employee_screen_interval',
              'resident_vaccination_ratio', 'employee_vaccination_ratio']

other_cols = [c for c in results.columns if c not in index_cols]
results = results[index_cols + other_cols]

dst = '../data/simulation_results'
results.to_csv(join(dst,'simulations_testing_and_vaccination_masks_{}.csv'\
                   .format(N_runs)), index=False)

results.head(3)

100%|██████████| 320/320 [2:52:04<00:00, 32.27s/it]  


CPU times: user 6.92 s, sys: 476 ms, total: 7.4 s
Wall time: 2h 52min 9s


,test_type,index_case,resident_screen_interval,employee_screen_interval,resident_vaccination_ratio,employee_vaccination_ratio,N_agents_0.025,N_agents_0.10,N_agents_0.25,N_agents_0.75,...,transmissions_std,undetected_infections_0.025,undetected_infections_0.10,undetected_infections_0.25,undetected_infections_0.75,undetected_infections_0.90,undetected_infections_0.975,undetected_infections_mean,undetected_infections_median,undetected_infections_std
0,same_day_antigen,employee,2.0,2.0,0.9,0.5,53.0,53.0,53.0,53.0,...,0.210437,0.0,0.0,0.0,0.0,0.0,1.0,0.0590,0.0,0.236496
1,same_day_antigen,employee,3.0,2.0,0.5,0.0,53.0,53.0,53.0,53.0,...,0.579451,0.0,0.0,0.0,0.0,0.0,1.0,0.0570,0.0,0.233585
2,same_day_antigen,employee,2.0,2.0,0.5,0.0,53.0,53.0,53.0,53.0,...,0.534400,0.0,0.0,0.0,0.0,0.0,1.0,0.0566,0.0,0.234537
